# Setup

In [1]:
import numpy as np
import pandas as pd
import torch

# Preprocessing

## Load 1 Hz data

In [15]:
hz_dataset_path = 'data/WEEE Dataset/processed/merged_weee_dataset.csv'
hz_dataset = pd.read_csv(hz_dataset_path, parse_dates=['Time'])
hz_dataset

,Time,heart_rate,VO2[mL/kg/min],VO2[mL/min],t,Weight,Height,Gender,Age,subject_id,EE[kcal/min],EE[kcal/kg/min]
0,2021-12-03 16:58:53,94.690403,6.81,470.17,0,70.6,166,0,29,P01,2319.81878,33.60054
1,2021-12-03 16:58:54,92.744725,6.81,470.17,1,70.6,166,0,29,P01,2319.81878,33.60054
2,2021-12-03 16:58:55,95.996020,7.12,491.33,2,70.6,166,0,29,P01,2424.22222,35.13008
3,2021-12-03 16:58:56,95.313119,7.12,491.33,3,70.6,166,0,29,P01,2424.22222,35.13008
4,2021-12-03 16:58:57,90.710056,7.12,491.33,4,70.6,166,0,29,P01,2424.22222,35.13008
...,...,...,...,...,...,...,...,...,...,...,...,...
27890,2021-12-02 17:34:03,75.463083,29.63,2287.33,1794,77.2,175,1,33,P17,11285.68622,146.19442
27891,2021-12-02 17:34:04,77.541560,28.16,2174.17,1795,77.2,175,1,33,P17,10727.35478,138.94144
27892,2021-12-02 17:34:05,76.800000,28.16,2174.17,1796,77.2,175,1,33,P17,10727.35478,138.94144
27893,2021-12-02 17:34:06,126.901198,28.16,2174.17,1797,77.2,175,1,33,P17,10727.35478,138.94144


In [16]:
hz_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27895 entries, 0 to 27894
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Time             27895 non-null  datetime64[ns]
 1   heart_rate       27895 non-null  float64       
 2   VO2[mL/kg/min]   27895 non-null  float64       
 3   VO2[mL/min]      27895 non-null  float64       
 4   t                27895 non-null  int64         
 5   Weight           27895 non-null  float64       
 6   Height           27895 non-null  int64         
 7   Gender           27895 non-null  int64         
 8   Age              27895 non-null  int64         
 9   subject_id       27895 non-null  object        
 10  EE[kcal/min]     27895 non-null  float64       
 11  EE[kcal/kg/min]  27895 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(4), object(1)
memory usage: 2.6+ MB


## Load raw acceleration data

In [18]:
raw_acc_path = 'data/WEEE Dataset/processed/raw_acceleration.csv'
raw_acc_df = pd.read_csv(raw_acc_path, parse_dates=['Time'])
raw_acc_df

,X,Y,Z,Time,subject_id
0,20.0,0.0,60.0,2021-12-03 16:57:44.000000,P01
1,19.0,0.0,59.0,2021-12-03 16:57:44.031250,P01
2,19.0,1.0,61.0,2021-12-03 16:57:44.062500,P01
3,20.0,1.0,60.0,2021-12-03 16:57:44.093750,P01
4,20.0,0.0,60.0,2021-12-03 16:57:44.125000,P01
...,...,...,...,...,...
1895503,-42.0,35.0,-33.0,2021-12-02 17:43:02.968750,P17
1895504,-42.0,36.0,-32.0,2021-12-02 17:43:03.000000,P17
1895505,-43.0,37.0,-32.0,2021-12-02 17:43:03.031250,P17
1895506,-44.0,36.0,-33.0,2021-12-02 17:43:03.062500,P17


In [19]:
raw_acc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1895508 entries, 0 to 1895507
Data columns (total 5 columns):
 #   Column      Dtype         
---  ------      -----         
 0   X           float64       
 1   Y           float64       
 2   Z           float64       
 3   Time        datetime64[ns]
 4   subject_id  object        
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 72.3+ MB
